In [4]:
using LCIO
using StatsPlots
using LinearAlgebra: norm
using JLD2
using IterableTables
gr()

Plots.GRBackend()

In [5]:
fileList = filter(s->occursin(r"E250_SetA.Pmumuh2ss.Gwhizard-2_84.eL0.8\.pR0.3\.\d\d_DST.slcio", s),
    readdir("/nfs/dust/ilc/user/jstrube/StrangeHiggs/data/RecoLevel", join=true))

90-element Array{String,1}:
 "/nfs/dust/ilc/user/jstrube/StrangeHiggs/data/RecoLevel/E250_SetA.Pmumuh2ss.Gwhizard-2_84.eL0.8.pR0.3.10_DST.slcio"
 "/nfs/dust/ilc/user/jstrube/StrangeHiggs/data/RecoLevel/E250_SetA.Pmumuh2ss.Gwhizard-2_84.eL0.8.pR0.3.11_DST.slcio"
 "/nfs/dust/ilc/user/jstrube/StrangeHiggs/data/RecoLevel/E250_SetA.Pmumuh2ss.Gwhizard-2_84.eL0.8.pR0.3.12_DST.slcio"
 "/nfs/dust/ilc/user/jstrube/StrangeHiggs/data/RecoLevel/E250_SetA.Pmumuh2ss.Gwhizard-2_84.eL0.8.pR0.3.13_DST.slcio"
 "/nfs/dust/ilc/user/jstrube/StrangeHiggs/data/RecoLevel/E250_SetA.Pmumuh2ss.Gwhizard-2_84.eL0.8.pR0.3.14_DST.slcio"
 "/nfs/dust/ilc/user/jstrube/StrangeHiggs/data/RecoLevel/E250_SetA.Pmumuh2ss.Gwhizard-2_84.eL0.8.pR0.3.15_DST.slcio"
 "/nfs/dust/ilc/user/jstrube/StrangeHiggs/data/RecoLevel/E250_SetA.Pmumuh2ss.Gwhizard-2_84.eL0.8.pR0.3.16_DST.slcio"
 "/nfs/dust/ilc/user/jstrube/StrangeHiggs/data/RecoLevel/E250_SetA.Pmumuh2ss.Gwhizard-2_84.eL0.8.pR0.3.17_DST.slcio"
 "/nfs/dust/ilc/user/jstrube/Strange

In [6]:
#obtain all important info and store as an object using JLD2
#strange quarks (can be applied for any file however)
#note - kaon and pion momentum as measured by the detector
Momcut = 1.87/2
costhetacut = 1657.0/2084.68

Momsspion = Float64[]
TOFsspion = Float64[]
Momsskaon = Float64[]
TOFsskaon = Float64[]
Momsspionmc = Float64[]
Momsskaonmc = Float64[]
Cossskaon = Float64[]
Cossspion = Float64[]
Momvecsspion = []
Momvecsskaon = []
Ensskaon = []
Ensspion = []
function main()
    filenum = 0
    count=0
    for file in fileList
        numEvents = 0
        filenum+=1
        LCIO.open(file) do reader
        for (iEvent, event) in enumerate(reader)
            numEvents+=1
            rp = getCollection(event, "PandoraPFOs")
            mcp = getCollection(event, "MCParticle")
            recoLinks = getCollection(event, "RecoMCTruthLink")
            rel=LCIO.LCRelationNavigator(recoLinks)
            coll = getCollection(event, "ECalBarrelHits")
            coll2 = getCollection(event, "ECalEndcapHits")
            decode = CellIDDecoder(coll)
            cellinfo = Dict()
                for hit in coll
                    layer = decode(hit)["layer"]
                    if layer != 0
                        continue
                    end
                    for i=1:LCIO.getNMCContributions(hit)
                        mcparticle = LCIO.getParticleCont(hit, i)
                        cell = LCIO.getCellID0(hit)
                        if LCIO.getCharge(mcparticle)==0
                            continue
                        end
                        recoparticle = getRelatedFromObjects(rel, mcparticle)
                        #if length(getRelatedToObjects(rel, mcparticle))!= 1  continue end
                        if length(recoparticle) != 1 continue end
                        recoparticle = recoparticle[1]
                        mom = norm(getMomentum(recoparticle))
                        mom2 = norm(getMomentum(mcparticle))
                        TOF = LCIO.getTimeCont(hit,i)
                        Mom3 = getMomentum(mcparticle)
                        costheta = Mom3[3]/sqrt(Mom3[1]^2+Mom3[2]^2+Mom3[3]^2)
                        if i>1
                            if TOF<LCIO.getTimeCont(hit,i-1)
                                count+=1
                            end
                        end
                        if TOF>10 || mom>10 || mom2>10 || mom<Momcut || mom2<Momcut || costheta>costhetacut
                            continue
                        end
                        if !(haskey(cellinfo, cell))
                            push!(cellinfo, cell => (mcparticle, recoparticle, TOF, costheta))
                        else
                            if cellinfo[cell][3]>TOF
                                cellinfo[cell]=(mcparticle, recoparticle, TOF, costheta)
                            end
                        end
                        
                    end
                end
                for hit in coll2
                    layer = decode(hit)["layer"]
                    if layer != 0
                        continue
                    end
                    for i=1:LCIO.getNMCContributions(hit)
                        mcparticle = LCIO.getParticleCont(hit, i)
                        cell = LCIO.getCellID0(hit)
                        if LCIO.getCharge(mcparticle)==0
                            continue
                        end
                        recoparticle = getRelatedFromObjects(rel, mcparticle)
                        #if length(getRelatedToObjects(rel, mcparticle))!= 1  continue end
                        if length(recoparticle) != 1 continue end
                        recoparticle = recoparticle[1]
                        mom = norm(getMomentum(recoparticle))
                        mom2 = norm(getMomentum(mcparticle))
                        TOF = LCIO.getTimeCont(hit,i)
                        Mom3 = getMomentum(mcparticle)
                        costheta = Mom3[3]/sqrt(Mom3[1]^2+Mom3[2]^2+Mom3[3]^2)
                        if i>1
                            if TOF<LCIO.getTimeCont(hit,i-1)
                                count+=1
                            end
                        end
                        if TOF>10 || mom>10 || mom2>10 || costheta < costhetacut
                            continue
                        end
                        if !(haskey(cellinfo, cell))
                            push!(cellinfo, cell => (mcparticle, recoparticle, TOF, costheta))
                        else
                            if cellinfo[cell][3]>TOF
                                cellinfo[cell]=(mcparticle, recoparticle, TOF, costheta)
                            end
                        end
                        
                    end
                end
                infodict = Dict()
                for (key, value) in cellinfo
                    if !haskey(infodict, value[1]) 
                        push!(infodict, value[1] => (value[2],value[3], value[4]))
                    else
                        if infodict[value[1]][2]>value[3]
                             infodict[value[1]] = (value[2],value[3], value[4])
                        end
                    end
                end
                   for (key, value) in infodict
                        if getPDG(key)==211
                            push!(Momvecsspion, getMomentum(value[1]))
                            push!(Ensspion, getEnergy(value[1]))
                            push!(Momsspion, norm(getMomentum(value[1])))
                            push!(Momsspionmc, norm(getMomentum(key)))
                            push!(TOFsspion, value[2])
                            push!(Cossspion, value[3])
                        end
                        if getPDG(key)==321
                            push!(Momvecsskaon, getMomentum(value[1]))
                            push!(Ensskaon, getEnergy(value[1]))
                            push!(Momsskaon, norm(getMomentum(value[1])))
                            push!(Momsskaonmc, norm(getMomentum(key)))
                            push!(TOFsskaon, value[2])
                            push!(Cossskaon, value[3])
                        end
                    end
                
            end
            if numEvents % 100 == 0
                println("processed ", numEvents, " events")
            end
        end
        println("read  $(filenum) files")
        
    end
    println("count is", count)
end

@time main()   
@save ("Momsskaon") Momsskaon
@save ("Momsspion") Momsspion
@save ("TOFsspion") TOFsspion
@save ("TOFsskaon") TOFsskaon
@save ("Cossspion") Cossspion
@save ("Cossskaon") Cossskaon
@save ("Ensskaon") Ensskaon
@save ("Ensspion") Ensspion
@save ("Momvecsspion") Momvecsspion
@save ("Momvecsskaon") Momvecsskaon

processed 1000 events
read  1 files
processed 1000 events
read  2 files
processed 1000 events
read  3 files
processed 1000 events
read  4 files
processed 1000 events
read  5 files
processed 1000 events
read  6 files
processed 1000 events
read  7 files
processed 1000 events
read  8 files
processed 1000 events
read  9 files
processed 1000 events
read  10 files
processed 500 events
read  11 files
processed 500 events
read  12 files
processed 500 events
read  13 files
processed 1000 events
read  14 files
processed 1000 events
read  15 files
processed 1000 events
read  16 files
processed 1000 events
read  17 files
processed 1000 events
read  18 files
processed 1000 events
read  19 files
processed 1000 events
read  20 files
processed 1000 events
read  21 files
processed 1000 events
read  22 files
processed 1000 events
read  23 files
processed 500 events
read  24 files
processed 1000 events
read  25 files
processed 1000 events
read  26 files
processed 1000 events
read  27 files
processed 1000